In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import conda_installer
from rdkit import Chem
from deepchem.feat.graph_features import atom_features as get_atom_features
import rdkit
import pickle
import matplotlib.pyplot as plt
import deepchem as dc


2023-04-17 17:19:15.982104: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 17:19:16.017243: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 17:19:16.018099: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 17:19:16.715970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
PDBs_folds = pickle.load(open('PDBs_folds_5.pkl', 'rb'))
y_folds = pickle.load(open('y_folds_5.pkl', 'rb'))
x_add_folds = pickle.load(open('x_add_folds_5.pkl', 'rb'))

In [3]:
%cd ../../PGGCN

/home/ali/GBNN/PGGCN


In [15]:
import models.PGNN_model_entropy as pg
import importlib
importlib.reload(pg)

<module 'models.PGNN_model_entropy' from '/home/ali/GBNN/PGGCN/models/PGNN_model_entropy.py'>

In [16]:
hists = []
predicts_train = []
predicts_test = []
loss = []
K = 5
for k in range(K):
    X_train = []
    x_add_train = []
    for i in range(K):
        if i != k:
            X_train += PDBs_folds[i]
            x_add_train += x_add_folds[i]
    X_test = PDBs_folds[k]
    x_add_test = x_add_folds[i]
    y_train = np.concatenate(np.array([y_folds[i] for i in range(K) if i != k]), 0)
    y_test = np.array(y_folds[k])
    x_train_parsed, shape_train = pg.data_generator(X_train, x_add_train)
    x_test_parsed, shape_test = pg.data_generator(X_test, x_add_test)
    hs, m = pg.get_trained_model(x_train_parsed, np.array(y_train), shape_train, len(X_train))
    with open('weights_info/PGNN_PG_K%d.pkl' % k, 'wb') as f:
        pickle.dump(m.layers[0].W_list, f)
        f.close()
    predicts_train.append(m.predict(x_train_parsed))
    hists.append(hs)
    loss.append(pg.test_model(x_test_parsed, y_test, len(y_folds[k]), shape_test, m))
    predicts_test.append(x_test_parsed)
    

Epoch 1/25
1/1 [==============================] - 4s 4s/step - loss: 12.5548
Epoch 2/25
1/1 [==============================] - 0s 58ms/step - loss: 10.6969
Epoch 3/25
1/1 [==============================] - 0s 55ms/step - loss: 9.3816
Epoch 4/25
1/1 [==============================] - 0s 49ms/step - loss: 8.2798
Epoch 5/25
1/1 [==============================] - 0s 55ms/step - loss: 7.3429
Epoch 6/25
1/1 [==============================] - 0s 52ms/step - loss: 6.5773
Epoch 7/25
1/1 [==============================] - 0s 52ms/step - loss: 5.8884
Epoch 8/25
1/1 [==============================] - 0s 51ms/step - loss: 5.2993
Epoch 9/25
1/1 [==============================] - 0s 53ms/step - loss: 4.7978
Epoch 10/25
1/1 [==============================] - 0s 60ms/step - loss: 4.3854
Epoch 11/25
1/1 [==============================] - 0s 54ms/step - loss: 4.0546
Epoch 12/25
1/1 [==============================] - 0s 56ms/step - loss: 3.7996
Epoch 13/25
1/1 [==============================] - 0s 52ms/st

1/1 [==============================] - 1s 592ms/step - loss: 3.2451
Epoch 1/25
1/1 [==============================] - 5s 5s/step - loss: 11.7975
Epoch 2/25
1/1 [==============================] - 0s 51ms/step - loss: 9.9627
Epoch 3/25
1/1 [==============================] - 0s 56ms/step - loss: 8.6150
Epoch 4/25
1/1 [==============================] - 0s 53ms/step - loss: 7.5489
Epoch 5/25
1/1 [==============================] - 0s 51ms/step - loss: 6.6498
Epoch 6/25
1/1 [==============================] - 0s 50ms/step - loss: 5.9020
Epoch 7/25
1/1 [==============================] - 0s 51ms/step - loss: 5.2682
Epoch 8/25
1/1 [==============================] - 0s 59ms/step - loss: 4.7277
Epoch 9/25
1/1 [==============================] - 0s 63ms/step - loss: 4.2806
Epoch 10/25
1/1 [==============================] - 0s 58ms/step - loss: 3.9206
Epoch 11/25
1/1 [==============================] - 0s 60ms/step - loss: 3.6351
Epoch 12/25
1/1 [==============================] - 0s 61ms/step - loss: 3

In [16]:
m.layers[0].weights[0].shape

TensorShape([75, 64])

In [12]:
with open('PGNN_DD_K5_hists.pkl', 'wb') as f:
    pickle.dump(hists, f)

In [13]:
with open('PGNN_DD_K5_loss.pkl', 'wb') as f:
    pickle.dump(loss, f)

In [14]:
with open('PGNN_DD_K5_predicts_train.pkl', 'wb') as f:
    pickle.dump(predicts_train, f)

In [9]:
y_train.mean()

-0.8791333333333333